In [ ]:
# Install TensorFlow
# !pip install -q tensorflow-gpu==2.0.0-rc0

try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass

import tensorflow as tf
print(tf.__version__)

# More imports
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, \
  BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os


# Load in the data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# map inputs to (-1, +1) for better training
x_train, x_test = x_train / 255.0 * 2 - 1, x_test / 255.0 * 2 - 1
print("x_train.shape:", x_train.shape)

# Flatten the data
N, H, W = x_train.shape
D = H * W
x_train = x_train.reshape(-1, D)
x_test = x_test.reshape(-1, D)

# Dimensionality of the latent space (posso cambiarlo, è un hyperparameter)
latent_dim = 100

# Get the generator model
def build_generator(latent_dim):
  i = Input(shape=(latent_dim,))
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
  x = BatchNormalization(momentum=0.7)(x)
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.7)(x)
  x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.7)(x)
  x = Dense(D, activation='tanh')(x) #Notice that because our image pixels are centered to be between -1 +1

  model = Model(i, x)
  return model

# Get the discriminator model
def build_discriminator(img_size):
  i = Input(shape=(img_size,))
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
  x = Dense(1, activation='sigmoid')(x)
  model = Model(i, x)
  return model



In [ ]:
# Compile both models in preparation for training


# Build and compile the discriminator
# D = input dimensionality
discriminator = build_discriminator(D)
discriminator.compile(
    loss='binary_crossentropy',
    optimizer=Adam(0.0002, 0.5),
    metrics=['accuracy'])

# Build and compile the combined model
generator = build_generator(latent_dim)

# we need to create a combined model where an input goes through both a generator and discriminator.

# Create an input to represent noise sample from latent space
z = Input(shape=(latent_dim,))

# Pass noise through generator to get an image
# So even though the generator is a model object it can still be used like a function to go from input to output.
img = generator(z)

# Make sure only the generator is trained
discriminator.trainable = False

#Next we pass in our image variable, which was the output of the generator, through the discriminator model.
# it's an output prediction from our discriminator for our fake images 

# The true output is fake, but we label them real!
fake_pred = discriminator(img)

# Create the combined model object
combined_model = Model(z, fake_pred)

# Compile the combined model
combined_model.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))


In [ ]:
# Train the GAN


# Config
batch_size = 32
epochs = 30000
sample_period = 200 # every `sample_period` steps generate and save some data


# Create batch labels to use when calling train_on_batch
ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

# Store the losses
d_losses = []
g_losses = []

# Create a folder to store generated images
if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

In [ ]:
#PLOTTING to have some preview

# A function to generate a grid of random samples from the generator
# and save them to a file
def sample_images(epoch):
  rows, cols = 5, 5
  noise = np.random.randn(rows * cols, latent_dim)
  imgs = generator.predict(noise)

  # Rescale images between 0 - 1
  imgs = 0.5 * imgs + 0.5

  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(H, W), cmap='gray')
      axs[i,j].axis('off')
      idx += 1
  fig.savefig("gan_images/%d.png" % epoch)
  plt.close()

In [ ]:
# Main training loop
for epoch in range(epochs):
  ###########################
  ### Train discriminator ###
  ###########################
  
  #We need both real and fake images.
  #We can get real images by sampling from random indices from zero up to the number of 
  #samples in X_train
  #then we can get the actual images by indexing X_train at those indices.

  # Select a random batch of images
  idx = np.random.randint(0, x_train.shape[0], batch_size)
  real_imgs = x_train[idx]
  
  # Generate fake images
  #we create some random noise sampled from the standard normal in our latent space.
  noise = np.random.randn(batch_size, latent_dim)

  #Then we call generate.predict with this noise as input. This gives us our fake images.
  fake_imgs = generator.predict(noise)
  
  #we call discriminator.train on batch
  #first we pass in the real images and our ones vector to denote that these belong to the positive class.
  #We get back the loss and the accuracy.
  #Next we pass in our fake images and our zeros vector to denote that these belong to the negative class.
  #We again get back the loss and accuracy in order to calculate the overall loss in accuracy.
  #We're going to take the mean of these losses and accuracies.

  # Train the discriminator
  # both loss and accuracy are returned
  d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
  d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc  = 0.5 * (d_acc_real + d_acc_fake)
  
  
  #######################
  ### Train generator ###
  #######################
  
  #for the generator We need to fake images only so first we generate some more noise

  noise = np.random.randn(batch_size, latent_dim)
  g_loss = combined_model.train_on_batch(noise, ones)

  #we call combined.model that train on batch.
  #The input is the noise and the target is our vector of ones.
  #This is because we are trying to trick the discriminator into thinking that the images from the generator are real.
  
  #The following code has been added later probably:
  # do it again!
  noise = np.random.randn(batch_size, latent_dim)
  g_loss = combined_model.train_on_batch(noise, ones)
  

  # Save the losses
  d_losses.append(d_loss)
  g_losses.append(g_loss)
  
  if epoch % 100 == 0:
    print(f"epoch: {epoch+1}/{epochs}, d_loss: {d_loss:.2f}, \
      d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")
  
  if epoch % sample_period == 0:
    sample_images(epoch)

#so if we look at the accuracy values we can see that despite the discriminator training it never reaches high accuracy.
#This is of course because while the discriminator is improving so too is the generator and that's exactly
#what we want obviously because we want the generator to be able to create realistic images.

In [ ]:
plt.plot(g_losses, label='g_losses')
plt.plot(d_losses, label='d_losses')
plt.legend()

!ls gan_images

In [ ]:
from skimage.io import imread
a = imread('gan_images/0.png')
plt.imshow(a)


In [ ]:
a = imread('gan_images/1000.png')
plt.imshow(a)


In [ ]:
a = imread('gan_images/5000.png')
plt.imshow(a)


In [ ]:
a = imread('gan_images/10000.png')
plt.imshow(a)


In [ ]:
a = imread('gan_images/20000.png')
plt.imshow(a)


In [ ]:
a = imread('gan_images/29800.png')
plt.imshow(a)
